In [1]:
import time
import pandas as pd
import os
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm

# Preparations

In [2]:
# Load the data
base_path_train = "~/shared/data/project/training"

items_df = pd.read_csv(os.path.join(base_path_train, "item_features.csv"))
purchase_df = pd.read_csv(os.path.join(base_path_train, "train_purchases.csv"))
session_df = pd.read_csv(os.path.join(base_path_train, "train_sessions.csv"))

base_path_test = "~/shared/data/project/test"

test_df = pd.read_csv(os.path.join(base_path_test, "test_sessions.csv"))

In [3]:
# Extend the purchases and sessions dataframes with bought information
purchase_df_processed = purchase_df.copy()
purchase_df_processed["was_bought"] = 1

session_df_processed = session_df.copy()
session_df_processed["was_bought"] = 0

In [4]:
# Combine the purchases and sessions dataframes into one
df_processed = pd.concat([purchase_df_processed, session_df_processed]).sort_values(["session_id", "date"])
df_processed

,session_id,item_id,date,was_bought
1,3,9655,2020-12-18 21:19:48.093,0
0,3,9655,2020-12-18 21:25:00.373,0
0,3,15085,2020-12-18 21:26:47.986,1
2,13,15654,2020-03-13 19:35:27.136,0
1,13,18626,2020-03-13 19:36:15.507,1
...,...,...,...,...
4743804,4440001,19539,2020-10-30 23:37:09.46,0
4743815,4440001,20409,2020-10-30 23:37:20.658,0
4743818,4440001,27852,2020-10-30 23:39:55.186,0
4743806,4440001,20449,2020-10-30 23:40:28.149,0


In [5]:
# Add a new column "categorized_feature" to items_df which holds the category ID and the feature value together as one value 
items_df["categorized_feature"] = items_df["feature_category_id"] * 10000 + items_df["feature_value_id"]
items_df

,item_id,feature_category_id,feature_value_id,categorized_feature
0,2,56,365,560365
1,2,62,801,620801
2,2,68,351,680351
3,2,33,802,330802
4,2,72,75,720075
...,...,...,...,...
471746,28143,68,351,680351
471747,28143,55,390,550390
471748,28143,11,109,110109
471749,28143,73,91,730091


In [6]:
# The "categorized feature" should be pivoted in order to have one item with its features / row
items_processed_df = items_df.pivot_table(values='categorized_feature', index='item_id', columns='feature_category_id').reset_index()
items_processed_df.index.names = ['index']
column_list = [f"item_feature_{x+1}" for x in list(range(73))]

items_processed_df.columns = ["item_id"] + column_list
items_processed_df

,item_id,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,item_feature_8,item_feature_9,...,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72,item_feature_73
index,,,,,,,,,,,,,,,,,,,,,
0,2,NaN,NaN,NaN,NaN,NaN,NaN,70394.0,NaN,NaN,...,NaN,NaN,NaN,NaN,680351.0,690885.0,NaN,NaN,720075.0,NaN
1,3,NaN,NaN,30889.0,40618.0,50605.0,NaN,70452.0,NaN,NaN,...,NaN,650521.0,NaN,NaN,680014.0,690592.0,NaN,NaN,720075.0,730544.0
2,4,NaN,NaN,30793.0,40618.0,50605.0,NaN,70837.0,NaN,NaN,...,NaN,650521.0,NaN,NaN,680373.0,690538.0,NaN,NaN,720075.0,730544.0
3,7,NaN,NaN,NaN,NaN,NaN,NaN,70536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,680739.0,690592.0,NaN,NaN,720075.0,NaN
4,8,NaN,NaN,30793.0,40618.0,50605.0,NaN,70798.0,NaN,NaN,...,NaN,650521.0,NaN,NaN,680351.0,690592.0,NaN,NaN,720075.0,730544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,NaN,NaN,30793.0,40618.0,50605.0,NaN,70798.0,NaN,NaN,...,NaN,650521.0,NaN,NaN,680106.0,690805.0,NaN,NaN,720075.0,730544.0
23687,28140,NaN,20053.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,640080.0,NaN,NaN,670349.0,680351.0,NaN,NaN,710226.0,NaN,730544.0
23688,28141,10461.0,NaN,30889.0,40719.0,50605.0,NaN,70002.0,NaN,NaN,...,NaN,NaN,NaN,NaN,680379.0,690499.0,NaN,NaN,720075.0,730544.0


In [7]:
# The session data should be extended with the "categorized features"
# The goal is to compare the sessions based on the viewed items' features
df_processed2 = df_processed.merge(items_processed_df, how="left", on="item_id")
df_processed2["was_bought"] = df_processed2["was_bought"].astype(float)
df_processed2

,session_id,item_id,date,was_bought,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,...,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72,item_feature_73
0,3,9655,2020-12-18 21:19:48.093,0.0,NaN,20053.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,670349.0,680393.0,NaN,NaN,NaN,NaN,730544.0
1,3,9655,2020-12-18 21:25:00.373,0.0,NaN,20053.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,670349.0,680393.0,NaN,NaN,NaN,NaN,730544.0
2,3,15085,2020-12-18 21:26:47.986,1.0,NaN,20053.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,670349.0,680097.0,NaN,NaN,NaN,NaN,730544.0
3,13,15654,2020-03-13 19:35:27.136,0.0,NaN,NaN,NaN,40618.0,NaN,60766.0,...,NaN,650521.0,NaN,NaN,680351.0,690780.0,NaN,NaN,720219.0,NaN
4,13,18626,2020-03-13 19:36:15.507,1.0,NaN,NaN,30793.0,40618.0,50605.0,NaN,...,NaN,NaN,NaN,NaN,680739.0,690805.0,NaN,NaN,720075.0,730544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5743815,4440001,19539,2020-10-30 23:37:09.46,0.0,NaN,NaN,NaN,40618.0,NaN,NaN,...,NaN,NaN,NaN,NaN,680351.0,690885.0,NaN,NaN,720075.0,730544.0
5743816,4440001,20409,2020-10-30 23:37:20.658,0.0,NaN,NaN,NaN,40618.0,NaN,NaN,...,NaN,NaN,NaN,NaN,680351.0,690885.0,NaN,NaN,720075.0,730544.0
5743817,4440001,27852,2020-10-30 23:39:55.186,0.0,NaN,NaN,NaN,40618.0,NaN,60778.0,...,NaN,650550.0,NaN,NaN,680351.0,690362.0,NaN,NaN,720075.0,730544.0
5743818,4440001,20449,2020-10-30 23:40:28.149,0.0,NaN,NaN,NaN,40618.0,NaN,60778.0,...,NaN,650550.0,NaN,NaN,680351.0,690362.0,NaN,NaN,720075.0,730544.0


In [8]:
# Create a distinct list of item features per session to be able to compare the sessions based on the item features 
df_processed2_melted = df_processed2\
.melt(id_vars=['session_id'], value_vars=column_list)[["session_id","value"]]\
.dropna().drop_duplicates()

df_processed2_melted

,session_id,value
5,18,10461.0
26,19,10461.0
72,77,10461.0
126,119,10461.0
157,171,10461.0
...,...,...
419298830,4439994,730544.0
419298832,4439999,730091.0
419298833,4439999,730544.0
419298840,4440001,730544.0


In [9]:
# Prepare the test dataset
test_df_merged = test_df.merge(items_processed_df, how="left", on="item_id")
test_df_merged

,session_id,item_id,date,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,...,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72,item_feature_73
0,126,9655,2020-12-18 21:25:00.373,NaN,20053.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,670349.0,680393.0,NaN,NaN,NaN,NaN,730544.0
1,126,9655,2020-12-18 21:19:48.093,NaN,20053.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,670349.0,680393.0,NaN,NaN,NaN,NaN,730544.0
2,3234,13214,2020-09-20 20:10:42.039,10461.0,NaN,30793.0,40618.0,50605.0,NaN,70837.0,...,NaN,NaN,NaN,NaN,680031.0,690116.0,NaN,NaN,720075.0,730544.0
3,3234,13214,2020-09-20 20:11:53.966,10461.0,NaN,30793.0,40618.0,50605.0,NaN,70837.0,...,NaN,NaN,NaN,NaN,680031.0,690116.0,NaN,NaN,720075.0,730544.0
4,3234,3173,2020-09-20 20:06:13.217,NaN,NaN,NaN,40618.0,NaN,60539.0,70452.0,...,NaN,650610.0,NaN,NaN,680702.0,690499.0,NaN,NaN,720748.0,730544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197619,186456690,10471,2021-05-02 17:41:18.737,NaN,NaN,NaN,40618.0,NaN,NaN,70452.0,...,NaN,650521.0,NaN,NaN,680745.0,690592.0,NaN,NaN,720486.0,730091.0
197620,186456690,13385,2021-05-02 17:43:41.78,NaN,NaN,NaN,40618.0,NaN,NaN,70452.0,...,NaN,650521.0,NaN,NaN,680351.0,690592.0,NaN,NaN,720219.0,730091.0
197621,186456690,10471,2021-05-02 17:40:10.625,NaN,NaN,NaN,40618.0,NaN,NaN,70452.0,...,NaN,650521.0,NaN,NaN,680745.0,690592.0,NaN,NaN,720486.0,730091.0
197622,186456690,5382,2021-05-02 17:40:50.001,NaN,NaN,NaN,40505.0,NaN,NaN,70452.0,...,NaN,650521.0,NaN,NaN,680479.0,690592.0,NaN,NaN,720219.0,730091.0


In [10]:
# Create a distinct list of item features per session to be able to compare the sessions based on the item features 
test_df_melted = test_df_merged\
.melt(id_vars=['session_id'], value_vars=column_list)[["session_id","value"]]\
.dropna().drop_duplicates()

test_df_melted

,session_id,value
2,3234,10461.0
24,38850,10461.0
45,77742,10461.0
48,80388,10461.0
60,108486,10461.0
...,...,...
14426528,186453372,730091.0
14426539,186454716,730544.0
14426542,186456690,730091.0
14426544,186456690,730544.0


# Prototype approach with the session_id 3234

In [11]:
# The "target" session(s) are outer joined with the "train" sessions on the "categorized feature" column
joined = test_df_melted[test_df_melted["session_id"] == 3234].merge(df_processed2_melted, how="outer", on="value")
joined.rename(columns={'value': 'categorized_feature'}, inplace=True)
joined

,session_id_x,categorized_feature,session_id_y
0,3234.0,10461.0,18
1,3234.0,10461.0,19
2,3234.0,10461.0,77
3,3234.0,10461.0,119
4,3234.0,10461.0,171
...,...,...,...
46521165,NaN,730091.0,4439949
46521166,NaN,730091.0,4439964
46521167,NaN,730091.0,4439990
46521168,NaN,730091.0,4439999


In [12]:
# The joined dataframe will be grouped on the "train" sessions, so we will have the 
#  * count of the common distinct categorized features between "target" and "train" sessions ( = CCFta)
#  * count of the distinct categorized features in the "train" sessions ( = CFtr )
# the count of the distinct categorized features in the "target" session(s) are knew/should be done separately. ( = CFta)
#
# The similarity is calculated as "CCFta" / ("CFtr + CFta - CCFta") 
#
# The dataframe ordered by similarity

joined_grouped = joined.groupby(['session_id_y']).count()

joined_grouped.rename(columns={'session_id_x': 'count_common_features'}, inplace=True)
joined_grouped.rename(columns={'categorized_feature': 'count_features_id_y'}, inplace=True)

joined_grouped['session_id_x'] = 3234
joined_grouped['count_features_id_x'] = test_df_melted[test_df_melted["session_id"] == 3234].shape[0] 

joined_grouped['similarity'] = \
    joined_grouped['count_common_features'] \
    /   (joined_grouped['count_features_id_y'] \
        + joined_grouped['count_features_id_x'] \
        - joined_grouped['count_common_features']\
        )

joined_grouped.sort_values(by=['similarity'], inplace=True, ascending=False)
joined_grouped

,count_common_features,count_features_id_y,session_id_x,count_features_id_x,similarity
session_id_y,,,,,
2537426,35,40,3234,38,0.813953
1542302,33,36,3234,38,0.804878
1132005,35,41,3234,38,0.795455
2108400,34,39,3234,38,0.790698
3142549,33,38,3234,38,0.767442
...,...,...,...,...,...
1579767,0,18,3234,38,0.000000
985220,0,24,3234,38,0.000000
270710,0,5,3234,38,0.000000


In [13]:
# The first 100 rows will be taken from the ordered dataframe which holds the similarities between the "target" and "train" sessions

top100 = joined_grouped.head(100)
top100 = top100.reset_index()
top100.rename(columns={'session_id_y': 'session_id'}, inplace=True)
top100.insert(0, 'rank', range(1, 1 + len(top100)))
top100

,rank,session_id,count_common_features,count_features_id_y,session_id_x,count_features_id_x,similarity
0,1,2537426,35,40,3234,38,0.813953
1,2,1542302,33,36,3234,38,0.804878
2,3,1132005,35,41,3234,38,0.795455
3,4,2108400,34,39,3234,38,0.790698
4,5,3142549,33,38,3234,38,0.767442
...,...,...,...,...,...,...,...
95,96,2554583,29,36,3234,38,0.644444
96,97,4192691,29,36,3234,38,0.644444
97,98,202296,29,36,3234,38,0.644444
98,99,56943,34,49,3234,38,0.641509


In [14]:
# The top 100 similar sessions needs to be extended with the purchase informations

results = top100.merge(purchase_df_processed, how="inner", on="session_id")
results

,rank,session_id,count_common_features,count_features_id_y,session_id_x,count_features_id_x,similarity,item_id,date,was_bought
0,1,2537426,35,40,3234,38,0.813953,15502,2020-09-05 13:51:57.282,1
1,2,1542302,33,36,3234,38,0.804878,10569,2020-11-22 20:45:04.945,1
2,3,1132005,35,41,3234,38,0.795455,14394,2020-09-10 09:02:33.336,1
3,4,2108400,34,39,3234,38,0.790698,3173,2020-09-11 18:21:14.856,1
4,5,3142549,33,38,3234,38,0.767442,26853,2020-09-07 16:32:21.148,1
...,...,...,...,...,...,...,...,...,...,...
95,96,2554583,29,36,3234,38,0.644444,7871,2021-02-27 21:27:29.075,1
96,97,4192691,29,36,3234,38,0.644444,19712,2020-11-11 22:25:19.735,1
97,98,202296,29,36,3234,38,0.644444,6736,2020-10-23 19:15:22.174,1
98,99,56943,34,49,3234,38,0.641509,26853,2020-11-27 12:26:21.779,1


In [15]:
# The recommendations are

recommendation = results[['session_id_x','item_id','rank']].copy()
recommendation.columns = ['session_id','item_id','rank']
# recommendation
recommendation.to_csv("results_uucf.csv", index=False)

# Full approach

In [ ]:
# This approach runs out from memory

joined = test_df_melted.merge(df_processed2_melted, how="outer", on="value")


In [23]:
# This approach runs out of time as it would need about 166 hours to complete as each session needs 12 seconds, and there is 50k sessions

start = time.time()

li_df = []

sessions = test_df_melted.session_id.unique()

for session_id in sessions[:3]:
    joined = test_df_melted[test_df_melted['session_id'] == session_id].merge(df_processed2_melted, how="outer", on="value")
    joined.rename(columns={'value': 'categorized_feature'}, inplace=True)

    processed = joined.groupby(['session_id_y']).count()
    
    processed.rename(columns={'session_id_x': 'count_common_features'}, inplace=True)
    processed.rename(columns={'categorized_feature': 'count_features_id_y'}, inplace=True)
    
    processed['session_id_x'] = session_id
    processed['count_features_id_x'] = test_df_melted[test_df_melted["session_id"] == session_id].shape[0]

    processed['similarity'] = processed['count_common_features'] / (processed['count_features_id_y'] + test_df_melted[test_df_melted['session_id'] == session_id].shape[0] - processed['count_common_features'])
    processed.sort_values(by=['similarity'], inplace=True, ascending=False)
    
    top100 = processed.head(100)
    top100 = top100.reset_index()
    top100.rename(columns={'session_id_y': 'session_id'}, inplace=True)
    top100.insert(0, 'rank', range(1, 1 + len(top100)))
    
    tmp = top100.merge(purchase_df_processed, how="inner", on="session_id")
    
    li_df.append(tmp)

result = pd.concat(li_df, axis=0, ignore_index=True)
    
end = time.time()

print(end - start)

result[['session_id_x','item_id','rank']]

39.591227769851685


,session_id_x,item_id,rank
0,3234,15502,1
1,3234,10569,2
2,3234,14394,3
3,3234,3173,4
4,3234,26853,5
...,...,...,...
295,77742,27315,96
296,77742,27315,97
297,77742,22721,98
298,77742,24664,99


In [ ]:
# This approach runs out from disk space

df_processed2_melted.to_csv("yourdata2.csv")
df2_key = df_processed2_melted.value

# creating a empty bucket to save result
df_result = pd.DataFrame(columns=(test_df_melted.columns.append(df_processed2_melted.columns)).unique())
df_result.to_csv("df3.csv",index_label=False)

# deleting df2 to save memory
del(df_processed2_melted)

def preprocess(x):
    tmp=pd.merge(test_df_melted, x, how="outer", on="value")
    tmp.to_csv("df3.csv",mode="a",header=False,index=False)

reader = pd.read_csv("yourdata2.csv", chunksize=1000) # chunksize depends with you colsize

[preprocess(r) for r in reader]